# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 1 2022 11:21AM,249786,10,MSP215392,Methapharm-G,Released
1,Nov 1 2022 11:21AM,249786,10,MSP215396,Methapharm-G,Released
2,Nov 1 2022 11:21AM,249786,10,MSP215397,Methapharm-G,Released
3,Nov 1 2022 11:21AM,249786,10,MSP215398,"Methapharm, Inc.",Released
4,Nov 1 2022 11:21AM,249786,10,MSP215400,"Methapharm, Inc.",Released
5,Nov 1 2022 11:13AM,249783,10,CTF0011482,"Citieffe, Inc.",Released
6,Nov 1 2022 11:13AM,249783,10,CTF0011483,"Citieffe, Inc.",Released
7,Nov 1 2022 11:09AM,249782,10,Enova-10906,Emerginnova,Released
8,Nov 1 2022 11:09AM,249782,10,Enova-10907,Emerginnova,Released
9,Nov 1 2022 10:50AM,249781,16,8719147,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,249780,Released,1
28,249781,Released,1
29,249782,Released,2
30,249783,Released,2
31,249786,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249780,NaN,NaN,1.0
249781,NaN,NaN,1.0
249782,NaN,NaN,2.0
249783,NaN,NaN,2.0
249786,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249685,55.0,3.0,7.0
249712,0.0,0.0,1.0
249713,0.0,0.0,1.0
249716,2.0,1.0,0.0
249750,0.0,28.0,31.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249685,55,3,7
249712,0,0,1
249713,0,0,1
249716,2,1,0
249750,0,28,31


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249685,55,3,7
1,249712,0,0,1
2,249713,0,0,1
3,249716,2,1,0
4,249750,0,28,31


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249685,55,3,7
1,249712,,,1
2,249713,,,1
3,249716,2,1,
4,249750,,28,31


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 1 2022 11:21AM,249786,10,Methapharm-G
3,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc."
5,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc."
7,Nov 1 2022 11:09AM,249782,10,Emerginnova
9,Nov 1 2022 10:50AM,249781,16,Sartorius Bioprocess Solutions
10,Nov 1 2022 10:42AM,249780,20,"Exact-Rx, Inc."
11,Nov 1 2022 10:34AM,249779,10,Bio-PRF
12,Nov 1 2022 10:05AM,249777,10,Eywa Pharma Inc.
30,Nov 1 2022 9:54AM,249776,19,ACG North America LLC
31,Nov 1 2022 9:47AM,249774,12,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 1 2022 11:21AM,249786,10,Methapharm-G,,,5
1,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",,,5
2,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",,,2
3,Nov 1 2022 11:09AM,249782,10,Emerginnova,,,2
4,Nov 1 2022 10:50AM,249781,16,Sartorius Bioprocess Solutions,,,1
5,Nov 1 2022 10:42AM,249780,20,"Exact-Rx, Inc.",,,1
6,Nov 1 2022 10:34AM,249779,10,Bio-PRF,,,1
7,Nov 1 2022 10:05AM,249777,10,Eywa Pharma Inc.,,5,13
8,Nov 1 2022 9:54AM,249776,19,ACG North America LLC,,,1
9,Nov 1 2022 9:47AM,249774,12,"SD Head USA, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 11:21AM,249786,10,Methapharm-G,5,,
1,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",5,,
2,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",2,,
3,Nov 1 2022 11:09AM,249782,10,Emerginnova,2,,
4,Nov 1 2022 10:50AM,249781,16,Sartorius Bioprocess Solutions,1,,
5,Nov 1 2022 10:42AM,249780,20,"Exact-Rx, Inc.",1,,
6,Nov 1 2022 10:34AM,249779,10,Bio-PRF,1,,
7,Nov 1 2022 10:05AM,249777,10,Eywa Pharma Inc.,13,5,
8,Nov 1 2022 9:54AM,249776,19,ACG North America LLC,1,,
9,Nov 1 2022 9:47AM,249774,12,"SD Head USA, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 11:21AM,249786,10,Methapharm-G,5,,
1,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",5,,
2,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",2,,
3,Nov 1 2022 11:09AM,249782,10,Emerginnova,2,,
4,Nov 1 2022 10:50AM,249781,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 11:21AM,249786,10,Methapharm-G,5.0,NaN,NaN
1,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",5.0,NaN,NaN
2,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",2.0,NaN,NaN
3,Nov 1 2022 11:09AM,249782,10,Emerginnova,2.0,NaN,NaN
4,Nov 1 2022 10:50AM,249781,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 1 2022 11:21AM,249786,10,Methapharm-G,5.0,0.0,0.0
1,Nov 1 2022 11:21AM,249786,10,"Methapharm, Inc.",5.0,0.0,0.0
2,Nov 1 2022 11:13AM,249783,10,"Citieffe, Inc.",2.0,0.0,0.0
3,Nov 1 2022 11:09AM,249782,10,Emerginnova,2.0,0.0,0.0
4,Nov 1 2022 10:50AM,249781,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3746591,61.0,14.0,6.0
12,249774,1.0,0.0,0.0
15,499435,38.0,31.0,55.0
16,749254,2.0,1.0,2.0
19,249776,1.0,0.0,0.0
20,499548,1.0,5.0,0.0
21,749178,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3746591,61.0,14.0,6.0
1,12,249774,1.0,0.0,0.0
2,15,499435,38.0,31.0,55.0
3,16,749254,2.0,1.0,2.0
4,19,249776,1.0,0.0,0.0
5,20,499548,1.0,5.0,0.0
6,21,749178,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,14.0,6.0
1,12,1.0,0.0,0.0
2,15,38.0,31.0,55.0
3,16,2.0,1.0,2.0
4,19,1.0,0.0,0.0
5,20,1.0,5.0,0.0
6,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,12,Released,1.0
2,15,Released,38.0
3,16,Released,2.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,6.0,0.0,55.0,2.0,0.0,0.0,0.0
Executing,14.0,0.0,31.0,1.0,0.0,5.0,1.0
Released,61.0,1.0,38.0,2.0,1.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,55.0,2.0,0.0,0.0,0.0
1,Executing,14.0,0.0,31.0,1.0,0.0,5.0,1.0
2,Released,61.0,1.0,38.0,2.0,1.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,55.0,2.0,0.0,0.0,0.0
1,Executing,14.0,0.0,31.0,1.0,0.0,5.0,1.0
2,Released,61.0,1.0,38.0,2.0,1.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()